In [2]:
!pip install spotipy

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import time
import datetime
import os

# client_id='e08812d0ad874b53bb01e49ada6166ad'
# client_secret='9085378d43d24d6c823a92c0f2cb03b5'

# 민진님
client_id = "7a98ceda33624c9bb15bf7f877176e50"
client_secret = "74fb60bce9d048e3a9faa52427e6d0f5"

# # 세인님
# client_id = "05c96c56746e49b9b26e2ccf9837c7b7"
# client_secret = "98238151cd5e46df93049946ddd3673a"

# # 서브 1
# client_id = "55cc1bbf202640ee8a691ecab9be6e1d"
# client_secret = "ef892d15d19543f4840c5ff069e6503c"

# # 서브 2
# client_id = "37211c9678ae40e6aed26be454135b1e"
# client_secret = "031a960bb8bc4367a412f63d25f1f513"

# # 서브 3
# client_id = "4cda4cd5cf9a4877b55336bedbe21c9c"
# client_secret = "93bacd33bd494ec5bc76034f3705d502"

# # 서브 4
# client_id = "9e6e9e6622204dbe85e2553c9b026ed5"
# client_secret = "b7b96ff10fae4077ab8912e1fb9d1f6b"

# # 서브 5
# client_id = "8e564786163d4a66a8e8d844955b9c2d"
# client_secret = "7966e74a259d4fd9a3d689b54fd3d100"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
# 검토 완료 진행상황 표시

start_time = time.time()

temporary_df = pd.DataFrame()  
# temporary_df2 = pd.DataFrame()  
# temporary_df3 = pd.DataFrame() 

artist_cache = {}  # 아티스트 정보 캐시 생성 요청 제한을 피하기 위해

# year_list = ['2022']
# genre_list = ['k-pop']

# year_list = ['2023', '2022', '2021', '2020']
# genre_list = ['k-pop', 'acoustic', 'club', 'dance', 'edm', 'disco', 'electronic', 'happy', 'hip-hop','party', 'pop', 'r-n-b', 'rock', 'romance', 'sleep', 'soul']

year_list = ['2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010','2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000']
genre_list = ['k-pop', 'acoustic', 'club', 'dance', 'edm', 'disco', 'electronic', 'happy', 'hip-hop','party', 'pop', 'r-n-b', 'rock', 'romance', 'sleep', 'soul']


total_workload = len(year_list) * len(genre_list) * 100  
completed_workload = 0

for year in year_list:
    for genre in genre_list:
        print(f"{year}_{genre}: 다운로드")    
        
        artist_name = []
        track_name = []
        track_popularity = []
        artist_id = []
        track_id = []  
        release_date = []
        album_image = []

        artist_popularity = []
        artist_followers =[]

        track_features_list = []
        
        for i in range(0, 5, 5):
            track_results = sp.search(q=f'year:{year} genre:{genre}', type='track', limit=50, offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                artist_id.append(t['artists'][0]['id'])
                artist_name.append(t['artists'][0]['name'])
                track_id.append(t['id'])
                track_name.append(t['name'])
                release_date.append(t['album']['release_date'])
                track_popularity.append(t['popularity'])
                try:
                    album_image.append(t['album']['images'][1]['url'])
                except IndexError:
                    album_image.append(None)                
                print(f"음악트랙 데이터 수집 횟수 : {i+1} // 3초 대기")
                #time.sleep(3)

                # # 아티스트 정보를 요청 코드
                # artist_info = None
                # if t['artists'][0]['id'] in artist_cache:
                #     artist_info = artist_cache[t['artists'][0]['id']]
                # else:
                #     artist_info = sp.artist(t['artists'][0]['id'])
                #     artist_cache[t['artists'][0]['id']] = artist_info

                # artist_popularity.append(artist_info['popularity'])  
                # artist_followers.append(artist_info['followers']['total'])  
                # print(f"아티스트 데이터 수집 횟수 : {i+1} // 3초 대기")
                # time.sleep(3)

                # # 오디오 특성 요청 코드
                # track_features = sp.audio_features([t['id']])
                # if track_features:
                #     track_features_list.append(track_features[0])
                # else:
                #     track_features_list.append(None)
                # print(f"음악특성 데이터 수집 횟수 : {i+1} // 3초 대기")
                # time.sleep(3)
                
                completed_workload += 1  
                progress = (completed_workload / total_workload) * 100  
                print(f"전체 진행 상황: {progress/10:.2f}% ,,, {progress:.0f}개")

                
        track_df = pd.DataFrame({ 'artist_id': artist_id, 'artist_name': artist_name, 'track_id': track_id, 'track_name': track_name, 
                                 'release_date': release_date, 'track_popularity': track_popularity, 'album_image': album_image})

        # track_df2 = pd.DataFrame({'artist_id': artist_id, 'artist_name': artist_name, 'artist_popularity': artist_popularity, 'artist_followers': artist_followers,
        #                           'track_id': track_id})


        # track_df3 = pd.DataFrame(track_features_list)
        
        temporary_df = pd.concat([temporary_df, track_df])
        # temporary_df2 = pd.concat([temporary_df2, track_df2])  
        # temporary_df3 = pd.concat([temporary_df3, track_df3])

temporary_df.drop_duplicates(subset='track_id', inplace=True)

timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
#temporary_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/test_track.csv', index=False)
#temporary_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/API/track/{year}_{genre}_track_{timestamp}.csv', index=False)


# #temporary_df2.drop_duplicates(subset='artist_id', inplace=True)
# temporary_df2.drop_duplicates(subset='track_id', inplace=True)
# temporary_df2 = temporary_df2.drop(columns=['track_id'])

# timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
# #temporary_df2.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/test_artist.csv', index=False)
# temporary_df2.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/API/artist/{year}_{genre}_artist_{timestamp}.csv', index=False)

# temporary_df3.drop_duplicates(subset='id', inplace=True)
# temporary_df3 = temporary_df3.rename(columns={'id': 'track_id'})
# temporary_df3 = temporary_df3.rename(columns={'key': 'music_key'})
# temporary_df3 = temporary_df3.rename(columns={'mode': 'music_mode'})
# temporary_df3 = temporary_df3.drop(columns=['type', 'uri', 'track_href', 'analysis_url', 'duration_ms'])

# timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
# #temporary_df3.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/test_audio3.csv', index=False)
# temporary_df3.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/API/audio/{year}_{genre}_audio_{timestamp}.csv', index=False)

end_time = time.time()
print(f"작업 소요 시간: {end_time - start_time} 초")

# 2023년 k-pop 10개 뽑는데 대략 10분 * 100분  1000분 16시간???


2023_k-pop: 다운로드
음악트랙 데이터 수집 횟수 : 1 // 3초 대기
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 2 // 3초 대기
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 3 // 3초 대기
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 4 // 3초 대기
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 5 // 3초 대기
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 6 // 3초 대기
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 7 // 3초 대기
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 8 // 3초 대기
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 9 // 3초 대기
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 10 // 3초 대기
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 11 // 3초 대기
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 12 // 3초 대기
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 13 // 3초 대기
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 14 // 3초 대기
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 15 // 3초 대기
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 16 // 3초 대기
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 17 // 3초 대기
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 18 // 3초 대기
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 19 // 3초 대기
전체 진행 상황: 0.00% ,,, 0개
음악트랙

In [4]:
temporary_df.head(100)

,artist_id,artist_name,track_id,track_name,release_date,track_popularity,album_image
0,2LIk90788K0zvyj2JJVwkJ,Jack Harlow,4xhsWYTOGcal8zt0J161CU,Lovin On Me,2023-11-10,96.0,https://i.scdn.co/image/ab67616d00001e02fcf4ad...
1,6HaGTQPmzraVmaVxvz6EUc,Jung Kook,2KslE17cAJNHTsI2MI0jb2,Standing Next to You,2023-11-03,98.0,https://i.scdn.co/image/ab67616d00001e02741fd4...
2,1oSPZhvZMIrWW5I41kPkkY,Jimin,3Ua0m0YmEjrMi9XErKcNiR,Like Crazy,2023-03-24,94.0,https://i.scdn.co/image/ab67616d00001e022b4607...
3,4SpbR6yFEvexJuaBpgAU5p,LE SSERAFIM,74X2u8JMVooG2QbjRxXwR8,Perfect Night,2023-10-27,90.0,https://i.scdn.co/image/ab67616d00001e025e352f...
4,1oSPZhvZMIrWW5I41kPkkY,Jimin,58BWnoFDNaeM0vwA0K3fqb,Like Crazy,2023-03-26,90.0,https://i.scdn.co/image/ab67616d00001e02f9638e...
...,...,...,...,...,...,...,...
45,4phGZZrJZRo4ElhRtViYdl,Jason Mraz,5upen9QriGWOp7BGPweugK,Lucky,2023-11-23,0.0,https://i.scdn.co/image/ab67616d00001e02ff25b4...
46,4phGZZrJZRo4ElhRtViYdl,Jason Mraz,1XiLYnmflN5WcrlmlpEslp,I’m Yours,2023-12-01,0.0,https://i.scdn.co/image/ab67616d00001e029e2ff4...
47,4phGZZrJZRo4ElhRtViYdl,Jason Mraz,6pMABS0WnvQEJ62WxBf15H,I’m Yours,2023-12-02,0.0,https://i.scdn.co/image/ab67616d00001e0271c756...
48,4phGZZrJZRo4ElhRtViYdl,Jason Mraz,2dkIw7F1pYD0UcIi0NRRY9,I’m Yours,2023-12-01,1.0,https://i.scdn.co/image/ab67616d00001e02f01ed5...


In [ ]:
start_time = time.time()

temporary_df = pd.DataFrame()  

artist_cache = {}  


year_list = ['2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010','2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000']
genre_list = ['k-pop']


total_workload = len(year_list) * len(genre_list) * 100  
completed_workload = 0

for year in year_list:
    for genre in genre_list:
        print(f"{year}_{genre}: 다운로드")    
        
        artist_name = []
        track_name = []
        track_popularity = []
        artist_id = []
        track_id = []  
        release_date = []
        album_image = []

        artist_popularity = []
        artist_followers =[]

        track_features_list = []
        
        for i in range(0, 5, 5):
            track_results = sp.search(q=f'year:{year} genre:{genre}', type='track', limit=50, offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                artist_id.append(t['artists'][0]['id'])
                artist_name.append(t['artists'][0]['name'])
                track_id.append(t['id'])
                track_name.append(t['name'])
                release_date.append(t['album']['release_date'])
                track_popularity.append(t['popularity'])
                try:
                    album_image.append(t['album']['images'][1]['url'])
                except IndexError:
                    album_image.append(None)                
                print(f"음악트랙 데이터 수집 횟수 : {i+1} // 3초 대기")

                
                completed_workload += 1  
                progress = (completed_workload / total_workload) * 100  
                print(f"전체 진행 상황: {progress/10:.2f}% ,,, {progress:.0f}개")

                
        track_df = pd.DataFrame({ 'artist_id': artist_id, 'artist_name': artist_name, 'track_id': track_id, 'track_name': track_name, 
                                 'release_date': release_date, 'track_popularity': track_popularity, 'album_image': album_image})


        
        temporary_df = pd.concat([temporary_df, track_df])


temporary_df.drop_duplicates(subset='track_id', inplace=True)

timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
temporary_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/test_track.csv', index=False)
temporary_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/API/track/{year}_{genre}_track_{timestamp}.csv', index=False)



end_time = time.time()
print(f"작업 소요 시간: {end_time - start_time} 초")



In [ ]:
start_time = time.time()

# 모든 데이터를 저장할 빈 DataFrame 생성
temporary_df = pd.DataFrame()  

artist_cache = {}  

year_list = ['2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010','2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000']
genre_list = ['k-pop']

total_workload = len(year_list) * len(genre_list) * 100  
completed_workload = 0

for year in year_list:
    for genre in genre_list:
        print(f"{year}_{genre}: 다운로드")    
        
        artist_name = []
        track_name = []
        track_popularity = []
        artist_id = []
        track_id = []  
        release_date = []
        album_image = []

        artist_popularity = []
        artist_followers =[]

        track_features_list = []
        
        for i in range(0, 5, 5):
            track_results = sp.search(q=f'year:{year} genre:{genre}', type='track', limit=50, offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                artist_id.append(t['artists'][0]['id'])
                artist_name.append(t['artists'][0]['name'])
                track_id.append(t['id'])
                track_name.append(t['name'])
                release_date.append(t['album']['release_date'])
                track_popularity.append(t['popularity'])
                try:
                    album_image.append(t['album']['images'][1]['url'])
                except IndexError:
                    album_image.append(None)                
                print(f"음악트랙 데이터 수집 횟수 : {i+1} // 3초 대기")

                
                completed_workload += 1  
                progress = (completed_workload / total_workload) * 100  
                print(f"전체 진행 상황: {progress/10:.2f}% ,,, {progress:.0f}개")

                
        track_df = pd.DataFrame({ 'artist_id': artist_id, 'artist_name': artist_name, 'track_id': track_id, 'track_name': track_name, 
                                 'release_date': release_date, 'track_popularity': track_popularity, 'album_image': album_image})

        
        # 각 장르와 연도의 데이터를 temporary_df에 추가
        temporary_df_2 = pd.concat([temporary_df, track_df])

# 중복 데이터 제거
temporary_df_2.drop_duplicates(subset='track_id', inplace=True)

# 모든 for문을 실행한 후에 한 번만 csv 파일로 저장
# timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
# temporary_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/API/track/{genre}_total_track_{timestamp}.csv', index=False)

end_time = time.time()
print(f"작업 소요 시간: {end_time - start_time} 초")


In [ ]:
temporary_df_2.head(200)

In [ ]:
start_time = time.time()

artist_cache = {}  

year_list = ['2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010','2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000']
genre_list = ['k-pop', 'acoustic', 'club', 'dance', 'edm', 'disco', 'electronic', 'happy', 'hip-hop','party', 'pop', 'r-n-b', 'rock', 'romance', 'sleep', 'soul']

total_workload = len(year_list) * len(genre_list) * 100  
completed_workload = 0

for genre in genre_list:
    # 각 장르별로 데이터를 저장할 빈 DataFrame 생성
    temporary_df = pd.DataFrame()  

    for year in year_list:
        print(f"{year}_{genre}: 다운로드")    
        
        artist_name = []
        track_name = []
        track_popularity = []
        artist_id = []
        track_id = []  
        release_date = []
        album_image = []

        artist_popularity = []
        artist_followers =[]

        track_features_list = []
        
        for i in range(0, 5, 5):
            track_results = sp.search(q=f'year:{year} genre:{genre}', type='track', limit=50, offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                artist_id.append(t['artists'][0]['id'])
                artist_name.append(t['artists'][0]['name'])
                track_id.append(t['id'])
                track_name.append(t['name'])
                release_date.append(t['album']['release_date'])
                track_popularity.append(t['popularity'])
                try:
                    album_image.append(t['album']['images'][1]['url'])
                except IndexError:
                    album_image.append(None)                
                print(f"음악트랙 데이터 수집 횟수 : {i+1} // 3초 대기")

                
                completed_workload += 1  
                progress = (completed_workload / total_workload) * 100
                print(f"{year}_{genre}: 진행중") 
                print(f"전체 진행 상황: {progress/10:.2f}% ,,, {progress:.0f}개")

                
        track_df = pd.DataFrame({ 'artist_id': artist_id, 'artist_name': artist_name, 'track_id': track_id, 'track_name': track_name, 
                                 'release_date': release_date, 'track_popularity': track_popularity, 'album_image': album_image})

        
        # 각 연도의 데이터를 temporary_df에 추가
        temporary_df = pd.concat([temporary_df, track_df])

        # 중복 데이터 제거
        temporary_df.drop_duplicates(subset='track_id', inplace=True)

        # 모든 연도를 포함하는 각 장르별로 한 번만 csv 파일로 저장
        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        #temporary_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/API/track/{genre}_total_track_{timestamp}.csv', index=False)
        temporary_df.to_csv('./data/{genre}_total_track_{timestamp}.csv', index=False)

end_time = time.time()
print(f"작업 소요 시간: {end_time - start_time} 초")


In [ ]:
start_time = time.time()

artist_cache = {}  

year_list = ['2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010','2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000']
genre_list = ['k-pop', 'acoustic', 'club', 'dance', 'edm', 'disco', 'electronic', 'happy', 'hip-hop','party', 'pop', 'r-n-b', 'rock', 'romance', 'sleep', 'soul']

total_workload = len(year_list) * len(genre_list) * 100  
completed_workload = 0

for genre in genre_list:
    # 각 장르별로 데이터를 저장할 빈 DataFrame 생성
    temporary_df = pd.DataFrame()  

    for year in year_list:
        print(f"{year}_{genre}: 다운로드")    
        
        artist_name = []
        track_name = []
        track_popularity = []
        artist_id = []
        track_id = []  
        release_date = []
        album_image = []

        artist_popularity = []
        artist_followers =[]

        track_features_list = []
        
        for i in range(0, 5, 5):
            track_results = sp.search(q=f'year:{year} genre:{genre}', type='track', limit=50, offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                artist_id.append(t['artists'][0]['id'])
                artist_name.append(t['artists'][0]['name'])
                track_id.append(t['id'])
                track_name.append(t['name'])
                release_date.append(t['album']['release_date'])
                track_popularity.append(t['popularity'])
                try:
                    album_image.append(t['album']['images'][1]['url'])
                except IndexError:
                    album_image.append(None)                
                print(f"음악트랙 데이터 수집 횟수 : {i+1} // 3초 대기")

                
                completed_workload += 1  
                progress = (completed_workload / total_workload) * 100  
                print(f"전체 진행 상황: {progress/10:.2f}% ,,, {progress:.0f}개")

                
        track_df = pd.DataFrame({ 'artist_id': artist_id, 'artist_name': artist_name, 'track_id': track_id, 'track_name': track_name, 
                                 'release_date': release_date, 'track_popularity': track_popularity, 'album_image': album_image})

        
        # 각 연도의 데이터를 temporary_df에 추가
        temporary_df = pd.concat([temporary_df, track_df])

    # 중복 데이터 제거
    temporary_df.drop_duplicates(subset='track_id', inplace=True)

    # 모든 연도를 포함하는 각 장르별로 한 번만 csv 파일로 저장
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    #temporary_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/API/track/{genre}_{year}_total_track_{timestamp}.csv', index=False)
    temporary_df.to_csv(f'./data/{genre}_{year}_total_track_{timestamp}.csv', index=False)

end_time = time.time()
print(f"작업 소요 시간: {end_time - start_time} 초")


In [9]:
start_time = time.time()

artist_cache = {}  

year_list = ['2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010','2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000']
genre_list = ['edm', 'disco', 'electronic', 'happy', 'hip-hop','party', 'pop', 'r-n-b', 'rock', 'romance', 'sleep', 'soul']

total_workload = len(year_list) * len(genre_list) * 100  
completed_workload = 0

for genre in genre_list:
    # 각 장르별로 데이터를 저장할 빈 DataFrame 생성
    temporary_df = pd.DataFrame()  

    for year in year_list:
        print(f"{year}_{genre}: 다운로드")    
        
        artist_name = []
        track_name = []
        track_popularity = []
        artist_id = []
        track_id = []  
        release_date = []
        album_image = []

        artist_popularity = []
        artist_followers =[]

        track_features_list = []
        
        for i in range(0, 1000, 50):
            track_results = sp.search(q=f'year:{year} genre:{genre}', type='track', limit=50, offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                artist_id.append(t['artists'][0]['id'])
                artist_name.append(t['artists'][0]['name'])
                track_id.append(t['id'])
                track_name.append(t['name'])
                release_date.append(t['album']['release_date'])
                track_popularity.append(t['popularity'])
                try:
                    album_image.append(t['album']['images'][1]['url'])
                except IndexError:
                    album_image.append(None)                
                print(f"음악트랙 데이터 수집 횟수 : {i+1} // 3초 대기")

                
                completed_workload += 1  
                progress = (completed_workload / total_workload) * 100  
                print(f"{year}_{genre}: 진행중") 
                print(f"전체 진행 상황: {progress/10:.2f}% ,,, {progress:.0f}개")

                
        track_df = pd.DataFrame({ 'artist_id': artist_id, 'artist_name': artist_name, 'track_id': track_id, 'track_name': track_name, 
                                 'release_date': release_date, 'track_popularity': track_popularity, 'album_image': album_image})

        
        # 각 연도의 데이터를 temporary_df에 추가
        temporary_df = pd.concat([temporary_df, track_df])

    # 중복 데이터 제거
    temporary_df.drop_duplicates(subset='track_id', inplace=True)

    # 모든 연도를 포함하는 각 장르별로 한 번만 csv 파일로 저장
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    temporary_df.to_csv(f'./data/{genre}_total_track_{timestamp}.csv', index=False)

end_time = time.time()
print(f"작업 소요 시간: {end_time - start_time} 초")


2023_edm: 다운로드
음악트랙 데이터 수집 횟수 : 1 // 3초 대기
2023_edm: 진행중
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 2 // 3초 대기
2023_edm: 진행중
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 3 // 3초 대기
2023_edm: 진행중
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 4 // 3초 대기
2023_edm: 진행중
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 5 // 3초 대기
2023_edm: 진행중
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 6 // 3초 대기
2023_edm: 진행중
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 7 // 3초 대기
2023_edm: 진행중
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 8 // 3초 대기
2023_edm: 진행중
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 9 // 3초 대기
2023_edm: 진행중
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 10 // 3초 대기
2023_edm: 진행중
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 11 // 3초 대기
2023_edm: 진행중
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 12 // 3초 대기
2023_edm: 진행중
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 13 // 3초 대기
2023_edm: 진행중
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 14 // 3초 대기
2023_edm: 진행중
전체 진행 상황: 0.00% ,,, 0개
음악트랙 데이터 수집 횟수 : 15 // 3초 대기
2023_edm: 진행중
전체 진행 상황: 0.01% ,,, 0개
음악트랙

KeyboardInterrupt: 